In [1]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
from scipy.optimize import minimize

# Load test samples
from aeons.tools import pickle_in
samples_g_1, samples_c_1, samples_w_1 = pickle_in("../test_samples/samples_1.pickle")

from aeons.covariance import logX_mu, logX_covinv_chol, points_at_iteration, X_mu, X_Sigmainv, data_at_iteration
from aeons.true_distribution import generate_Xs
from aeons.lm_partial import analytic_lm_params

### Try on real likelihood model

In [ ]:
def full_logL_X(X, theta):
    logLmax, d, sigma = theta
    return logLmax - X**(2/d)/(2*sigma**2)

def full_X_logL(logL, theta):
    logLmax, d, sigma = theta
    return (2*sigma**2 * (logLmax - logL))**(d/2)

def full_logL_prime(X, theta):
    logLmax, d, sigma = theta
    return - 1/(d*sigma**2) * X**(2/d - 1)

def logPr_full(logL, mean, covinv, theta):
    Xstar = full_X_logL(logL, theta)
    return - 1/2 * (Xstar - mean).T @ covinv @ (Xstar - mean) 

def logPr_bayes_full(logL, mean, covinv, theta):
    Xstar = full_X_logL(logL, theta)
    log_abs_fprimes = np.log(abs(full_logL_prime(Xstar, theta)))
    return - np.sum(log_abs_fprimes) - 1/2 * (Xstar - mean).T @ covinv @ (Xstar - mean)

def minimise_bayes_full(logL, mean, covinv, x0):
    def func(theta):
        return - logPr_bayes_full(logL, mean, covinv, theta)
    solution = minimize(func, x0, bounds=((logL.max(), np.inf), (0, np.inf), (0, np.inf)))
    return solution

def minimise_gaussian_full(logL, mean, covinv, x0):
    def func(theta):
        return - logPr_full(logL, mean, covinv, theta)
    solution = minimize(func, x0)
    return solution

In [ ]:
nk, logL = data_at_iteration(samples_g_1, 8000)
mean_X = X_mu(nk)
covinv_X = X_Sigmainv(nk)
covinv_ls = np.eye(len(mean_X))

In [ ]:
theta_l = analytic_lm_params(logL, mean_X, 10)
theta_b = minimise_bayes_full(logL, mean_X, covinv_X, theta_l).x
print(theta_b, theta_l)

[-0.19023726  9.76205936  0.10020569] [-0.3088430612631896, 9.72222790754946, 0.10080065717795862]


In [ ]:
logPr_bayes_full(logL, mean_X, covinv_X, theta_b)

-6243.92316118433